In [4]:
from collections import defaultdict

def get_knight_moves(pos, n):
    """Get all valid knight moves from position (row, col) on an n×n board."""
    row, col = pos
    moves = []
    knight_offsets = [(-2, -1), (-2, 1), (-1, -2), (-1, 2), 
                      (1, -2), (1, 2), (2, -1), (2, 1)]
    
    for dr, dc in knight_offsets:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < n and 0 <= new_col < n:
            moves.append((new_row, new_col))
    
    return moves

def build_knight_graph(n):
    """Build adjacency list representation of n×n knight graph."""
    graph = defaultdict(list)
    
    for row in range(n):
        for col in range(n):
            pos = (row, col)
            neighbors = get_knight_moves(pos, n)
            graph[pos] = neighbors
    
    return graph

def can_two_cops_win(graph, cop1_start, cop2_start, robber_start, max_depth=15):
    """
    Check if 2 cops can guarantee winning against robber starting at robber_start.
    Uses minimax with the cops trying to minimize robber's options.
    """
    def get_reachable_positions(pos):
        """Get positions reachable in one move (including staying put)."""
        return [pos] + graph[pos]
    
    # Memoization for game states
    memo = {}
    
    def minimax(cop1_pos, cop2_pos, robber_pos, is_cops_turn, depth):
        """
        Minimax search. Returns True if cops can guarantee win from this state.
        """
        if depth > max_depth:
            return False  # Assume robber escapes if game goes too long
        
        # Check if robber is caught
        if robber_pos == cop1_pos or robber_pos == cop2_pos:
            return True
        
        state = (cop1_pos, cop2_pos, robber_pos, is_cops_turn, depth)
        if state in memo:
            return memo[state]
        
        if is_cops_turn:
            # Cops move - they win if ANY move leads to a winning position
            cop1_moves = get_reachable_positions(cop1_pos)
            cop2_moves = get_reachable_positions(cop2_pos)
            
            for new_cop1_pos in cop1_moves:
                for new_cop2_pos in cop2_moves:
                    if minimax(new_cop1_pos, new_cop2_pos, robber_pos, False, depth + 1):
                        memo[state] = True
                        return True
            
            memo[state] = False
            return False
        
        else:
            # Robber moves - cops win if ALL robber moves lead to cop wins
            robber_moves = get_reachable_positions(robber_pos)
            
            for new_robber_pos in robber_moves:
                # Skip if robber would move to cop position (invalid)
                if new_robber_pos == cop1_pos or new_robber_pos == cop2_pos:
                    continue
                    
                if not minimax(cop1_pos, cop2_pos, new_robber_pos, True, depth + 1):
                    memo[state] = False
                    return False
            
            memo[state] = True
            return True
    
    return minimax(cop1_start, cop2_start, robber_start, True, 0), None

def test_two_cops_diagonal(n):
    """Test if 2 cops can win with one at center and one diagonally adjacent."""
    print(f"\nTesting 2 cops on {n}×{n} knight graph:")
    
    graph = build_knight_graph(n)
    all_positions = list(graph.keys())
    center = (n//2, n//2)
    
    # Place second cop one square diagonally from center
    diagonal_pos = (center[0] + 1, center[1] + 1)
    if diagonal_pos not in all_positions:
        # Try other diagonal if that one is off the board
        diagonal_pos = (center[0] - 1, center[1] - 1)
    
    print(f"Graph has {len(all_positions)} vertices")
    print(f"Cop 1 position (center): {center}")
    print(f"Cop 2 position (diagonal): {diagonal_pos}")
    print(f"Center connects to: {graph[center]}")
    print(f"Diagonal connects to: {graph[diagonal_pos]}")
    
    # Test this specific configuration against all robber starting positions
    print(f"\nTesting cops at {center} and {diagonal_pos} against all robber positions...")
    
    winning_robber_starts = []
    losing_robber_starts = []
    
    for robber_start in all_positions:
        if robber_start != center and robber_start != diagonal_pos:
            print(f"  Testing robber starting at {robber_start}... ", end="")
            
            can_win, _ = can_two_cops_win(graph, center, diagonal_pos, robber_start)
            
            if can_win:
                winning_robber_starts.append(robber_start)
                print("✓ Cops win")
            else:
                losing_robber_starts.append(robber_start)
                print("✗ Robber escapes")
    
    print(f"\nRESULTS:")
    print(f"Robber positions where cops win: {len(winning_robber_starts)}")
    print(f"Robber positions where robber escapes: {len(losing_robber_starts)}")
    
    if losing_robber_starts:
        print(f"\n✗ 2 cops CANNOT guarantee winning on {n}×{n} knight graph")
        print(f"Robber can escape when starting from these positions:")
        for pos in losing_robber_starts[:10]:  # Show first 10
            print(f"  {pos}")
        if len(losing_robber_starts) > 10:
            print(f"  ... and {len(losing_robber_starts) - 10} more positions")
        return False
    else:
        print(f"\n✓ 2 cops CAN guarantee winning on {n}×{n} knight graph!")
        print(f"With cops at {center} and {diagonal_pos}, robber cannot escape from any starting position")
        return True

def main():
    """Test whether 2 cops are sufficient for 5×5 and 6×6 knight graphs."""
    
    print("Testing cop number for knight graphs")
    print("We know 3 cops is sufficient - testing if 2 cops is sufficient")
    print("Placing first cop at center, second cop one square diagonally")
    print("Doing brute force analysis on all robber starting positions")
    
    # Test 5×5
    print("=" * 60)
    result_5x5 = test_two_cops_diagonal(5)
    
    # Test 6×6
    print("\n" + "=" * 60)
    result_6x6 = test_two_cops_diagonal(6)
    
    # Summary
    print("\n" + "=" * 60)
    print("FINAL RESULTS:")
    print(f"5×5 knight graph: Cop number is {'2' if result_5x5 else '3'}")
    print(f"6×6 knight graph: Cop number is {'2' if result_6x6 else '3'}")

if __name__ == "__main__":
    main()

Testing cop number for knight graphs
We know 3 cops is sufficient - testing if 2 cops is sufficient
Placing first cop at center, second cop one square diagonally
Doing brute force analysis on all robber starting positions

Testing 2 cops on 5×5 knight graph:
Graph has 25 vertices
Cop 1 position (center): (2, 2)
Cop 2 position (diagonal): (3, 3)
Center connects to: [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)]
Diagonal connects to: [(1, 2), (1, 4), (2, 1), (4, 1)]

Testing cops at (2, 2) and (3, 3) against all robber positions...
  Testing robber starting at (0, 0)... ✓ Cops win
  Testing robber starting at (0, 1)... ✓ Cops win
  Testing robber starting at (0, 2)... ✓ Cops win
  Testing robber starting at (0, 3)... ✓ Cops win
  Testing robber starting at (0, 4)... ✓ Cops win
  Testing robber starting at (1, 0)... ✓ Cops win
  Testing robber starting at (1, 1)... ✓ Cops win
  Testing robber starting at (1, 2)... ✓ Cops win
  Testing robber starting at (1, 3)... ✓ Cops 